# Modelo Xception



In [ ]:
# Librerías
import random
import pandas as pd 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, Adamax, Nadam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [ ]:
# Montamos el directorio Drive para cargar las imágenes
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Parámetros de las imágenes
HEIGHT = 224
WIDTH = 224
CHANNELS = 3
N_CLASSES = 29
TRAIN_DIR = "/content/gdrive/MyDrive/Backup/DATCOM/AA/Sentinel2LULC_354/Sentinel2LULC_354/"
AUG_TRAIN_DIR = "/content/gdrive/MyDrive/Backup/DATCOM/AA/Sentinel2LULC_354_augmented/Sentinel2LULC_354_augmented/"
TEST_DIR = "/content/gdrive/MyDrive/Backup/DATCOM/AA/Test 2"

# Parámetros de entrenamiento
BATCH_SIZE = 64
EPOCHS = 100 #10 #25 #35
LEARNING_RATE = 1e-4 #1e-3 #1e-4 #1e-5
LOSS = 'categorical_crossentropy'
# Métricas a mostrar en las gráficas
METRICS = ['accuracy', 'AUC']

## Conjunto de entrenamiento original


In [ ]:
# Leemos las imágenes de entrenamiento y las dividimos en 80% para entrenamiento
# del modelo y 20% para validación 
train_datagenerator = ImageDataGenerator(rescale=1/255, validation_split=0.2)
train_it = train_datagenerator.flow_from_directory(directory=TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH),
                                                    class_mode='categorical',
                                                    batch_size=BATCH_SIZE,
                                                    subset='training')
validation_it = train_datagenerator.flow_from_directory(directory=TRAIN_DIR,
                                                        target_size=(HEIGHT, WIDTH),
                                                        class_mode='categorical',
                                                        batch_size=BATCH_SIZE,
                                                        subset='validation')

## Data Augmentation

In [ ]:
# Leemos las imágenes de entrenamiento y las dividimos en 80% para entrenamiento
# del modelo y 20% para validación 
train_datagenerator = ImageDataGenerator(rescale=1/255, 
                                         validation_split=0.2,
                                         rotation_range=45,
                                         width_shift_range=.15,
                                         height_shift_range=.15,
                                         horizontal_flip=True,
                                         vertical_flip=True,
                                         zoom_range=0.5)
train_it = train_datagenerator.flow_from_directory(directory=TRAIN_DIR,
                                                    target_size=(HEIGHT, WIDTH),
                                                    class_mode='categorical',
                                                    batch_size=BATCH_SIZE,
                                                    subset='training')
validation_it = train_datagenerator.flow_from_directory(directory=TRAIN_DIR,
                                                        target_size=(HEIGHT, WIDTH),
                                                        class_mode='categorical',
                                                        batch_size=BATCH_SIZE,
                                                        subset='validation')

## Data Augmentation + Conjunto de entrenamiento original

In [ ]:
# Leemos las imágenes de entrenamiento y las dividimos en 80% para entrenamiento
# del modelo y 20% para validación 
train_datagenerator = ImageDataGenerator(rescale=1/255, validation_split=0.2)
train_it = train_datagenerator.flow_from_directory(directory=AUG_TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH),
                                                    class_mode='categorical',
                                                    batch_size=BATCH_SIZE,
                                                    subset='training')
validation_it = train_datagenerator.flow_from_directory(directory=AUG_TRAIN_DIR,
                                                        target_size=(HEIGHT, WIDTH),
                                                        class_mode='categorical',
                                                        batch_size=BATCH_SIZE,
                                                        subset='validation')

## Carga del modelo

In [ ]:
# Cargamos el modelo con los pesos del entrenamiento sobre ImageNet
base_model = Xception(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, CHANNELS))
print("Nº de capas: ", len(base_model.layers))

### Usando los pesos originales

In [ ]:
# PRIMERA VERSIÓN: sin re-entrenar ninguna capa
base_model.trainable = False

### Re-entrenando algunas capas

In [ ]:
# SEGUNDA VERSIÓN: finetuning re-entrenando algunas capas
base_model.trainable = True
# Número de capas a re-entrenar
fine_tune_at = 100 #10 #50
# Congelamos los pesos de todas las capas anteriores a las que se van a re-entrenar
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

### Entrenando el modelo completo

In [ ]:
# TERCERA VERSIÓN: re-entrenando todas las capas
base_model.trainable = True

## Adaptación del modelo

### Tradicional

In [ ]:
# Adaptamos el modelo a nuestro problema
input_layer = Input(shape=(HEIGHT, WIDTH, CHANNELS), name='input_image')
output_conv = base_model(input_layer)
x = Flatten(name='flatten')(output_conv) 
# Última capa de salida con las 29 clases
x = Dense(N_CLASSES, activation='softmax', name='classes')(x)

### Regularización L1

In [ ]:
# Adaptamos el modelo a nuestro problema
input_layer = Input(shape=(HEIGHT, WIDTH, CHANNELS), name='input_image')
output_conv = base_model(input_layer)
x = Flatten(name='flatten')(output_conv) 
# Última capa de salida con las 29 clases
x = Dense(N_CLASSES, activation='softmax', name='classes', kernel_regularizer=l1(0.1))(x)

### Regularización L2

In [ ]:
# Adaptamos el modelo a nuestro problema
input_layer = Input(shape=(HEIGHT, WIDTH, CHANNELS), name='input_image')
output_conv = base_model(input_layer)
x = Flatten(name='flatten')(output_conv) 
# Última capa de salida con las 29 clases
x = Dense(N_CLASSES, activation='softmax', name='classes', kernel_regularizer=l2(0.1))(x)

### Regularización L1 y L2

In [ ]:
# Adaptamos el modelo a nuestro problema
input_layer = Input(shape=(HEIGHT, WIDTH, CHANNELS), name='input_image')
output_conv = base_model(input_layer)
x = Flatten(name='flatten')(output_conv) 
# Última capa de salida con las 29 clases
x = Dense(N_CLASSES, activation='softmax', name='classes', kernel_regularizer=l1_l2(l1=0.1, l2=0.1))(x)

## Compilación del modelo

#### Optimizador Adam

In [ ]:
model = Model(inputs=input_layer, outputs=x)
opt = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=opt, loss=LOSS, metrics=METRICS)

#### Optimizador Adamax

In [ ]:
model = Model(inputs=input_layer, outputs=x)
opt = Adamax(learning_rate=LEARNING_RATE)
model.compile(optimizer=opt, loss=LOSS, metrics=METRICS)

#### Optimizador Nadam

In [ ]:
model = Model(inputs=input_layer, outputs=x)
opt = Nadam(learning_rate=LEARNING_RATE)
model.compile(optimizer=opt, loss=LOSS, metrics=METRICS)

## Entrenamiento del modelo

### Entrenamiento tradicional

In [ ]:
history = model.fit(train_it,
          validation_data=validation_it,
          steps_per_epoch=train_it.n // BATCH_SIZE,
          validation_steps=validation_it.n // BATCH_SIZE,
          epochs=EPOCHS,
          workers=100,
          use_multiprocessing=True)
# Guardamos el modelo en un fichero
model_filename = "xception_model.h5"
model.save(model_filename)

### Early Stopping

In [ ]:
history = model.fit(train_it,
          validation_data=validation_it,
          steps_per_epoch=train_it.n // BATCH_SIZE,
          validation_steps=validation_it.n // BATCH_SIZE,
          callbacks=[EarlyStopping(
            monitor="val_accuracy", 
            patience=10, 
            mode="auto",
            restore_best_weights=True)],
          epochs=EPOCHS,
          workers=100,
          use_multiprocessing=True)
# Guardamos el modelo en un fichero
model_filename = "xception_model.h5"
model.save(model_filename)

### Gráfica de la evolución del entrenamiento

In [ ]:
# Gráfica de evolución de la tasa de acierto durante entrenamiento y validación
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

## Predicciones sobre test

In [ ]:
# Cargamos las imágenes de test
test_datagenerator = ImageDataGenerator(rescale=1./255)
test_it = test_datagenerator.flow_from_directory(directory=TEST_DIR,
                                                  target_size=(HEIGHT, WIDTH),
                                                  batch_size= 1,
                                                  class_mode=None,
                                                  shuffle=False)
# Reseteamos el iterador sobre las imágenes de test
test_it.reset()

# Predecimos con el modelo entrenado sobre las imágenes de test
test_probs = model.predict(test_it, steps=test_it.samples)
test_preds = np.argmax(test_probs, axis=1)

# Construimos un traductor entre las etiquetas automáticamente generadas por Keras
# y las reales a partir de los nombres de las carpetas de entrenamiento
translated_train_labels = {train_it.class_indices[key]:int(key[:key.index("_")]) 
                        for key in train_it.class_indices}
# Traducimos las etiquetas de test resultantes a las reales
translated_test_preds = [translated_train_labels[label] for label in test_preds]

# Creamos el fichero de subida
id_jpg_column = [test_img[(test_img.find("/") + 1): ] for test_img in test_it.filenames]
preds_dataset = pd.DataFrame({'id.jpg': id_jpg_column, 'label': translated_test_preds})
preds_filename = "xception_submission.csv"
preds_dataset.to_csv(preds_filename, index=False)